In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import operators
import lowlevelheuristics

In [2]:
def fitness(results):
    fitnesss = []
    lengths = []
    for idx, result in enumerate(results):
        fitnesss.append(np.sum(np.square(np.array(result) / BIN_CAPACITYS[idx])) / len(result))
        lengths.append(len(result))
    
    return np.mean(fitnesss)

In [3]:
def applyChromosome(problemsItems, chromosome):
    results = []
    
    for pIdx, items in enumerate(problemsItems):
        result = []
        idx = 0
        prev = 0
        while idx < len(items):
            for heuristic in chromosome:
                if idx < len(items):
                    result, prev = heuristic(result, items[idx], BIN_CAPACITYS[pIdx], prev)
                    idx += 1
                else:
                    break
        results.append(result)
            
    return results

In [4]:
def generateChromosome():
    chromosome = []
    chromosomeLength = np.random.randint(low=MIN_CHROMOSOME_SIZE,high=MAX_CHROMOSOME_SIZE)
    for i in range(chromosomeLength):
        heuristicChoice = np.random.randint(len(heuristicList))
        chromosome.append(heuristicList[heuristicChoice])
        
    return chromosome

In [5]:
def createPopulation():
    population = []
    for i in range(POPULATION_SIZE):
        population.append(generateChromosome())
        
    return population

In [6]:
def getPopulationFitness(population):
    populationResults = []
    for individual in population:
        populationResults.append(applyChromosome(problemsItems, individual))
        
    populationFitness = []
    for idx, result in enumerate(populationResults):
        populationFitness.append(fitness(result))
    
    return np.array(populationFitness), populationResults

In [7]:
def tournamentSelector(population, tournament_size=5):
    # Make the tournament
    random_indicies = np.random.randint(len(population), size=tournament_size).tolist()
    tournament = []
    for idx, val in np.ndenumerate(random_indicies):
        tournament.append(population[val])

    # Run the tournament
    fitnesss, results = getPopulationFitness(tournament)
    
    maxPos = np.argmax(fitnesss, axis=0)
    minPos = np.argmin(fitnesss, axis=0)
    
    return [random_indicies[maxPos], fitnesss[maxPos]], [random_indicies[minPos], fitnesss[minPos]]

A genetic algorithm that mutates by doing swap operations 2 times and does a crossover operation at 2 points

In [8]:
def geneticAlgorithm(population):
    newPop = population[:]
    
    for i in range(NUM_MUTATE):    
        best, worst = tournamentSelector(population, tournament_size=TOURNAMENT_SIZE)
        newPop[worst[0]] = operators.swap(population[best[0]], 1)
        
    for i in range(NUM_CROSSOVER):    
        best1, worst1 = tournamentSelector(population, tournament_size=TOURNAMENT_SIZE)
        best2, worst2 = tournamentSelector(population, tournament_size=TOURNAMENT_SIZE)
        result1, result2 = operators.crossover(population[best1[0]], population[best2[0]], 1)
        newPop[worst1[0]] = result1
        newPop[worst2[0]] = result2
    
    return newPop

In [9]:
def geneticAlgorithm2(population):
    newPop = []
    
    for i in range(NUM_MUTATE):    
        best, worst = tournamentSelector(population, tournament_size=TOURNAMENT_SIZE)
        newPop.append(operators.swap(population[best[0]], 1))
        
    for i in range(NUM_CROSSOVER):    
        best1, worst1 = tournamentSelector(population, tournament_size=TOURNAMENT_SIZE)
        best2, worst2 = tournamentSelector(population, tournament_size=TOURNAMENT_SIZE)
        result1, result2 = operators.crossover(population[best1[0]], population[best2[0]], 1)
        newPop.append(result1)
        newPop.append(result2)
        
    for i in range(NUM_REPRODUCTION):    
        best, worst = tournamentSelector(population, tournament_size=TOURNAMENT_SIZE)
        newPop.append(population[best[0]])
    
    return newPop

In [10]:
def tabooSearch(current):
    operator = operatorList[np.random.randint(len(operatorList))]
    new = operator(current, heuristicList, 1)
    
    if new in tabooList:
        return current
    if fitness(applyChromosome(problemsItems, new)) > fitness(applyChromosome(problemsItems, current)):
        current = new
    
    tabooList.append(new)
    
    return current

In [11]:
location = './problems/'

problems = [
            ['bin1data/N1C1W1_O.BPP', 32],
            ['bin1data/N1C1W1_P.BPP', 26],
            ['bin1data/N1C1W1_T.BPP', 28],
            ['bin1data/N1C1W1_G.BPP', 25],
            ['bin1data/N1C1W1_K.BPP', 26],
            ['bin1data/N1C1W1_S.BPP', 28],
            ['bin1data/N1C1W1_F.BPP', 27],
    
            ['bin2data/N1W1B1R0.BPP', 18],
            ['bin2data/N1W1B1R1.BPP', 18],
            ['bin2data/N1W2B3R2.BPP', 10],
            ['bin2data/N1W2B2R0.BPP', 10],
            
            ['bin3data/HARD1.BPP', 57],
            ['bin3data/HARD3.BPP', 57],
            ['bin3data/HARD4.BPP', 57],
            ['bin3data/HARD6.BPP', 57],
        ]

heuristicList = [
    lowlevelheuristics.firstFit,
    lowlevelheuristics.lastFit,
    lowlevelheuristics.nextFit,
    lowlevelheuristics.bestFit,
    lowlevelheuristics.worstFit,
    #lowlevelheuristics.randomFit
]

problemIndexs = [6]
problemName = 'p1.png'

# problemIndexs = [0, 1, 2, 3, 4, 5]
# problemName = 'p2.png'

# problemIndexs = [10]
# problemName = 'p3.png'

# problemIndexs = [7, 8, 9]
# problemName = 'p4.png'

# problemIndexs = [14]
# problemName = 'p5.png'

# problemIndexs = [11, 12, 13]
# problemName = 'p6.png'


problemsItems = []
PROBLEM_SIZES = []
BIN_CAPACITYS = []
for idx, problemIndex in enumerate(problemIndexs):
    problemSet = pd.read_csv(location + problems[problemIndex][0], header=None).values.tolist()

    PROBLEM_SIZES.append(problemSet.pop(0)[0])
    BIN_CAPACITYS.append(problemSet.pop(0)[0])

    items = pd.DataFrame(problemSet)
    items = np.array(items[0])
    items = -np.sort(-items)

    assert PROBLEM_SIZES[idx] == len(items)
    
    problemsItems.append(items)

MIN_CHROMOSOME_SIZE = 10
MAX_CHROMOSOME_SIZE = max(PROBLEM_SIZES)

In [ ]:
POPULATION_SIZE = 25
GENERATIONS = 300
TOURNAMENT_SIZE = 3

MUTATION_RATE = 0.5
CROSSOVER_RATE = 0.3

NUM_MUTATE = int(MUTATION_RATE * POPULATION_SIZE)
NUM_CROSSOVER = int(CROSSOVER_RATE * POPULATION_SIZE)
NUM_REPRODUCTION = POPULATION_SIZE - (NUM_MUTATE + NUM_CROSSOVER)

assert NUM_REPRODUCTION >= 0

SAMPLES = 30

REPORT_RATE = 1

samplesGaOverTime = []
samplesGaFinal = []

print("Genetic Algorithm:")
for sample in range(SAMPLES):
    gaOverTime = []
    population = createPopulation()
    for gen in range(GENERATIONS):    
        population = geneticAlgorithm(population)

        if gen % REPORT_RATE == 0:
            fitnesss, results = getPopulationFitness(population)
            max = np.argmax(fitnesss, axis=0)
            gaOverTime.append(fitnesss[max])
            
    samplesGaOverTime.append(gaOverTime)
    fitnesss, results = getPopulationFitness(population)
    finalFitness = fitnesss[np.argmax(fitnesss, axis=0)]
    samplesGaFinal.append(finalFitness)
    print("Sample " + str(sample + 1) + ": " + str(finalFitness))

Genetic Algorithm:
Sample 1: 0.9705071428571428
Sample 2: 0.9588111111111112
Sample 3: 0.9521518518518519
Sample 4: 0.9651357142857142
Sample 5: 0.9635785714285715
Sample 6: 0.957392857142857
Sample 7: 0.9556689655172415
Sample 8: 0.9630148148148147
Sample 9: 0.9588111111111112
Sample 10: 0.9565296296296296
Sample 11: 0.9588111111111112
Sample 12: 0.9593035714285714
Sample 13: 0.9587962962962963
Sample 14: 0.9603344827586207
Sample 15: 0.9588111111111112
Sample 16: 0.9705214285714285
Sample 17: 0.9624666666666668
Sample 18: 0.9668241379310345
Sample 19: 0.9715310344827586
Sample 20: 0.9609571428571428
Sample 21: 0.9587962962962963
Sample 22: 0.9630148148148147
Sample 23: 0.9600620689655174
Sample 24: 0.9587962962962963
Sample 25: 0.9588111111111112


In [ ]:
operatorList = [
    operators.add,
    operators.change,
    operators.remove,
    operators.swap
]

tabooList = []

samplesTabuOverTime = []
samplesTabuFinal = []

print("Tabu Search:")
for sample in range(SAMPLES):
    tabuOverTime = []
    current = generateChromosome()
    for gen in range(GENERATIONS):    
        current = tabooSearch(current)

        if gen % REPORT_RATE == 0:
            result = fitness(applyChromosome(problemsItems, current))
            tabuOverTime.append(result)
            
    samplesTabuOverTime.append(tabuOverTime)
    finalFitness = fitness(applyChromosome(problemsItems, current))
    samplesTabuFinal.append(finalFitness)
    print("Sample " + str(sample + 1) + ": " + str(finalFitness))

In [ ]:
print('Low level heuristics:')

lowLevelFinal = []

nonRandomHeuristicList = heuristicList[:len(heuristicList)]

print(nonRandomHeuristicList)

for heuristic in nonRandomHeuristicList:
    heuristicFit = fitness(applyChromosome(problemsItems, [heuristic]))
    lowLevelFinal.append(heuristicFit)

In [ ]:
print('Random fit heuristics:')

samplesRandomFinal = []

for sample in range(SAMPLES):
    heuristicFit = fitness(applyChromosome(problemsItems, [lowlevelheuristics.randomFit]))
    samplesRandomFinal.append(heuristicFit)
    print("Sample " + str(sample + 1) + ": " + str(heuristicFit))

In [ ]:
print(samplesRandomFinal)
generations = np.arange(int(GENERATIONS/REPORT_RATE))

meanGaOverTime = np.mean(samplesGaOverTime, axis=0)
meanTabuOverTime = np.mean(samplesTabuOverTime, axis=0)

meanGaFinal = np.mean(samplesGaFinal)
meanTabuFinal = np.mean(samplesTabuFinal)
meanRandomFinal = np.mean(samplesRandomFinal)

stdGaFinal = np.std(samplesGaFinal)
stdTabuFinal = np.std(samplesTabuFinal)
stdRandomFinal = np.std(samplesRandomFinal)

maxGaFinal = np.max(samplesGaFinal)
maxTabuFinal = np.max(samplesTabuFinal)
maxRandomFinal = np.max(samplesRandomFinal)

print("Genetic Algorithm Final: " + str(meanGaFinal))
print("Tabu Search Final: " + str(meanTabuFinal))
print("Random Final: " + str(meanRandomFinal))

fig = plt.figure()
plt.grid(1)
plt.xlim([0, GENERATIONS])
plt.ion()
plt.xlabel('Generations')
plt.ylabel('Fitness')

plots = []
descriptions = []
plots.append(plt.plot(generations, meanGaOverTime, 'b--', linewidth=1, markersize=3)[0])
plots.append(plt.plot(generations, meanTabuOverTime, 'r--', linewidth=1, markersize=3)[0])
descriptions.append("Genetic algorithm")
descriptions.append("Tabu search")

plt.legend(plots, descriptions)
fig.savefig(problemName)
plt.show(5)

plt.close()

In [ ]:
algNames = ['Genetic algorithm', 'Tabu search', 'Random fit', 'First fit', 'Last fit', 'Next fit', 'Best fit', 'Worst fit']
meanFinal = [meanGaFinal, meanTabuFinal, meanRandomFinal]
meanFinal.extend(lowLevelFinal)

padding = [None, None, None, None, None]

stdFinal =[stdGaFinal, stdTabuFinal, stdRandomFinal]
stdFinal.extend(padding)

maxFinal =[maxGaFinal, maxTabuFinal, maxRandomFinal]
maxFinal.extend(padding)

firstSample = 49
algFitFirst = [meanGaOverTime[firstSample], meanTabuOverTime[firstSample], None]
algFitFirst.extend(padding)

secondSample = 149
algFitSecond = [meanGaOverTime[secondSample], meanTabuOverTime[secondSample], None]
algFitSecond.extend(padding)


d = {'Empty': pd.to_numeric([None, None, None, None, None, None, None, None]), 
     'Algorithm': algNames, 
     '50': algFitFirst, 
     '150': algFitSecond, 
     'mean': meanFinal,
     'std': stdFinal,
     'max': maxFinal
    }

df = pd.DataFrame(data=d)

list(df.columns.values)

result = df[['Empty', 'Algorithm', '50', '150', 'mean', 'std', 'max']]
result

In [ ]:
print(result.to_latex(index=False, bold_rows=True, na_rep=''))